---
sidebar_position: 3
---

# 如何创建一个自定义LLM类

```{=mdx}
:::info 预备知识

本指南假定您已经了解以下概念：

- [LLMs](/docs/concepts/text_llms)

:::
```

这个笔记本介绍如何创建一个自定义LLM包装器，以防您希望使用自己的LLM或使用一个LangChain未直接支持的包装器。

自定义LLM在继承 [`LLM` 类](https://api.js.langchain.com/classes/langchain_core.language_models_llms.LLM.html) 后，需要实现以下必要内容：

- 一个 `_call` 方法，接收一个字符串和调用选项（包括诸如 `stop` 序列等内容），并返回一个字符串。
- 一个 `_llmType` 方法，返回一个字符串。仅用于日志记录。

您还可以实现以下可选方法：

- 一个 `_streamResponseChunks` 方法，返回一个 `AsyncIterator` 并逐个产生 [`GenerationChunks`](https://api.js.langchain.com/classes/langchain_core.outputs.GenerationChunk.html)。这使得LLM能够支持流式输出。

让我们实现一个非常简单的自定义LLM，它仅回显输入的前 `n` 个字符。

In [1]:
import { LLM, type BaseLLMParams } from "@langchain/core/language_models/llms";
import type { CallbackManagerForLLMRun } from "@langchain/core/callbacks/manager";
import { GenerationChunk } from "@langchain/core/outputs";

interface CustomLLMInput extends BaseLLMParams {
  n: number;
}

class CustomLLM extends LLM {
  n: number;

  constructor(fields: CustomLLMInput) {
    super(fields);
    this.n = fields.n;
  }

  _llmType() {
    return "custom";
  }

  async _call(
    prompt: string,
    options: this["ParsedCallOptions"],
    runManager: CallbackManagerForLLMRun
  ): Promise<string> {
    // Pass `runManager?.getChild()` when invoking internal runnables to enable tracing
    // await subRunnable.invoke(params, runManager?.getChild());
    return prompt.slice(0, this.n);
  }

  async *_streamResponseChunks(
    prompt: string,
    options: this["ParsedCallOptions"],
    runManager?: CallbackManagerForLLMRun
  ): AsyncGenerator<GenerationChunk> {
    // Pass `runManager?.getChild()` when invoking internal runnables to enable tracing
    // await subRunnable.invoke(params, runManager?.getChild());
    for (const letter of prompt.slice(0, this.n)) {
      yield new GenerationChunk({
        text: letter,
      });
      // Trigger the appropriate callback
      await runManager?.handleLLMNewToken(letter);
    }
  }
}

我们现在可以像使用其他任何LLM一样使用它：

In [2]:
const llm = new CustomLLM({ n: 4 });

await llm.invoke("I am an LLM");

I am


支持流式传输：

In [3]:
const stream = await llm.stream("I am an LLM");

for await (const chunk of stream) {
  console.log(chunk);
}

I
 
a
m


## 更丰富的输出

如果你想利用 LangChain 的回调系统来实现诸如令牌追踪之类的功能，可以继承 [`BaseLLM`](https://api.js.langchain.com/classes/langchain_core.language_models_llms.BaseLLM.html) 类并实现更底层的
`_generate` 方法。该方法的输入和输出不再只是单个字符串，而是可以接受多个输入字符串，并将每个输入字符串映射为多个字符串输出。
此外，它返回的是一个包含附加元数据字段的 `Generation` 输出，而不仅仅是字符串。

In [4]:
import { CallbackManagerForLLMRun } from "@langchain/core/callbacks/manager";
import { LLMResult } from "@langchain/core/outputs";
import {
  BaseLLM,
  BaseLLMCallOptions,
  BaseLLMParams,
} from "@langchain/core/language_models/llms";

interface AdvancedCustomLLMCallOptions extends BaseLLMCallOptions {}

interface AdvancedCustomLLMParams extends BaseLLMParams {
  n: number;
}

class AdvancedCustomLLM extends BaseLLM<AdvancedCustomLLMCallOptions> {
  n: number;

  constructor(fields: AdvancedCustomLLMParams) {
    super(fields);
    this.n = fields.n;
  }

  _llmType() {
    return "advanced_custom_llm";
  }

  async _generate(
    inputs: string[],
    options: this["ParsedCallOptions"],
    runManager?: CallbackManagerForLLMRun
  ): Promise<LLMResult> {
    const outputs = inputs.map((input) => input.slice(0, this.n));
    // Pass `runManager?.getChild()` when invoking internal runnables to enable tracing
    // await subRunnable.invoke(params, runManager?.getChild());

    // One input could generate multiple outputs.
    const generations = outputs.map((output) => [
      {
        text: output,
        // Optional additional metadata for the generation
        generationInfo: { outputCount: 1 },
      },
    ]);
    const tokenUsage = {
      usedTokens: this.n,
    };
    return {
      generations,
      llmOutput: { tokenUsage },
    };
  }
}

这会将额外的返回信息传递到回调事件和 `streamEvents` 方法中：

In [5]:
const llm = new AdvancedCustomLLM({ n: 4 });

const eventStream = await llm.streamEvents("I am an LLM", {
  version: "v2",
});

for await (const event of eventStream) {
  if (event.event === "on_llm_end") {
    console.log(JSON.stringify(event, null, 2));
  }
}

{
  "event": "on_llm_end",
  "data": {
    "output": {
      "generations": [
        [
          {
            "text": "I am",
            "generationInfo": {
              "outputCount": 1
            }
          }
        ]
      ],
      "llmOutput": {
        "tokenUsage": {
          "usedTokens": 4
        }
      }
    }
  },
  "run_id": "a9ce50e4-f85b-41eb-bcbe-793efc52f9d8",
  "name": "AdvancedCustomLLM",
  "tags": [],
  "metadata": {}
}
